In [1]:
import argparse
import itertools
import json
import math
import os
import sys
import timeit

import dolfinx
import gmsh
import h5py
import matplotlib.pyplot as plt
import meshio
import multiphenicsx
import multiphenicsx.fem
import multiphenicsx.fem.petsc
import numpy as np
import petsc4py
import pyvista
import pyvista as pv
import pyvistaqt as pvqt
import subprocess
import ufl
import viskex
import warnings

from basix.ufl import element, mixed_element
from dolfinx import cpp, default_real_type, default_scalar_type, fem, io, la, mesh, nls, plot
from dolfinx.fem import petsc
from dolfinx.io import gmshio, VTXWriter, XDMFFile
from dolfinx.nls import petsc as petsc_nls
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
from IPython.display import Image

from mpi4py import MPI
from petsc4py import PETSc
from ufl import (Circumradius, FacetNormal, SpatialCoordinate, TrialFunction, TestFunction,
                 dot, div, dx, ds, dS, grad, inner, grad, avg, jump)

import commons, configs, geometry, utils

warnings.simplefilter("ignore")

In [2]:
def arctanh(y):
    return 0.5 * ufl.ln((1 + y) / (1 - y))

def ocv(c, cmax=35000):
    xi = 2 * (c - 0.5 * cmax) / cmax
    return -0.5 * arctanh(xi) + 3.25

In [3]:
comm = MPI.COMM_WORLD
encoding = io.XDMFFile.Encoding.HDF5
micron = 1e-6
dimensions = "150-40-0"
mesh_folder = "output/tertiary_current/150-40-0/20-55-20/5.0e-06/"
LX, LY, LZ = [float(vv) * micron for vv in dimensions.split("-")]
workdir = os.path.join(mesh_folder, "tertiary")
utils.make_dir_if_missing(workdir)
output_meshfile = os.path.join(mesh_folder, 'mesh.msh')
lines_h5file = os.path.join(mesh_folder, 'lines.h5')
potential_resultsfile = os.path.join(workdir, "potential.bp")
concentration_resultsfile = os.path.join(workdir, "concentration.bp")
concentration_xdmf_file = os.path.join(workdir, "concentration.xdmf")
# current_dist_file = os.path.join(workdir, f"current-y-positions-{str(args.Wa_p)}-{str(args.kr)}.png")
# reaction_dist_file = os.path.join(workdir, f"reaction-dist-{str(args.Wa_p)}-{str(args.kr)}.png")
current_resultsfile = os.path.join(workdir, "current.bp")
simulation_metafile = os.path.join(workdir, "simulation.json")

markers = commons.Markers()

# ### Read input geometry
partitioner = mesh.create_cell_partitioner(mesh.GhostMode.shared_facet)
domain, ct, ft = gmshio.read_from_msh(output_meshfile, comm, partitioner=partitioner)
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(tdim, fdim)
domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim)

x = SpatialCoordinate(domain)

# tag internal facets as 0
ft_imap = domain.topology.index_map(fdim)
num_facets = ft_imap.size_local + ft_imap.num_ghosts
indices = np.arange(0, num_facets)
values = np.zeros(indices.shape, dtype=np.intc)

values[ft.indices] = ft.values

ft = mesh.meshtags(domain, fdim, indices, values)
ct = mesh.meshtags(domain, tdim, ct.indices, ct.values)

dx = ufl.Measure("dx", domain=domain, subdomain_data=ct, metadata={"quadrature_degree": 4})
ds = ufl.Measure("ds", domain=domain, subdomain_data=ft, metadata={"quadrature_degree": 4})

f_to_c = domain.topology.connectivity(fdim, tdim)
c_to_f = domain.topology.connectivity(tdim, fdim)
charge_xfer_facets = ft.find(markers.electrolyte_v_positive_am)

other_internal_facets = ft.find(0)
int_facet_domain = []
for f in charge_xfer_facets:
    if f >= ft_imap.size_local or len(f_to_c.links(f)) != 2:
        continue
    c_0, c_1 = f_to_c.links(f)[0], f_to_c.links(f)[1]
    subdomain_0, subdomain_1 = ct.values[[c_0, c_1]]
    local_f_0 = np.where(c_to_f.links(c_0) == f)[0][0]
    local_f_1 = np.where(c_to_f.links(c_1) == f)[0][0]
    if subdomain_0 > subdomain_1:
        int_facet_domain.append(c_0)
        int_facet_domain.append(local_f_0)
        int_facet_domain.append(c_1)
        int_facet_domain.append(local_f_1)
    else:
        int_facet_domain.append(c_1)
        int_facet_domain.append(local_f_1)
        int_facet_domain.append(c_0)
        int_facet_domain.append(local_f_0)

other_internal_facet_domains = []
for f in other_internal_facets:
    if f >= ft_imap.size_local or len(f_to_c.links(f)) != 2:
        continue
    c_0, c_1 = f_to_c.links(f)[0], f_to_c.links(f)[1]
    subdomain_0, subdomain_1 = ct.values[[c_0, c_1]]
    local_f_0 = np.where(c_to_f.links(c_0) == f)[0][0]
    local_f_1 = np.where(c_to_f.links(c_1) == f)[0][0]
    other_internal_facet_domains.append(c_0)
    other_internal_facet_domains.append(local_f_0)
    other_internal_facet_domains.append(c_1)
    other_internal_facet_domains.append(local_f_1)
int_facet_domains = [(markers.electrolyte_v_positive_am, int_facet_domain)]#, (0, other_internal_facet_domains)]

dS = ufl.Measure("dS", domain=domain, subdomain_data=int_facet_domains)

Info    : Reading 'output/tertiary_current/150-40-0/20-55-20/5.0e-06/mesh.msh'...
Info    : 55 entities
Info    : 5106 nodes
Info    : 10380 elements
Info    : Done reading 'output/tertiary_current/150-40-0/20-55-20/5.0e-06/mesh.msh'


In [4]:
# viskex.dolfinx.plot_mesh(domain)

In [5]:
viskex.dolfinx.plot_mesh_tags(domain, ct, "subdomains")

Widget(value='<iframe src="http://localhost:45407/index.html?ui=P_0x71949ff6cc80_0&reconnect=auto" class="pyvi…

In [6]:
viskex.dolfinx.plot_mesh_tags(domain, ft, "boundaries and interfaces")

Widget(value='<iframe src="http://localhost:45407/index.html?ui=P_0x7194a7b00bc0_1&reconnect=auto" class="pyvi…

In [7]:
cells_Omega1 = ct.find(markers.electrolyte)
cells_Omega2 = ct.find(markers.positive_am)
facets_partial_Omega = np.hstack([ft.find(marker) for marker in [markers.left, markers.right, markers.insulated_electrolyte, markers.insulated_positive_am]])
facets_partial_Omega.sort()
facets_Gamma = ft.find(markers.electrolyte_v_positive_am)
facets_Gamma.shape, facets_partial_Omega.shape

((170,), (360,))

In [8]:
# Define function spaces
V = dolfinx.fem.functionspace(domain, ("Lagrange", 2))

V1 = V.clone()
V2 = V.clone()
M = V.clone()

In [9]:
# Define restrictions
dofs_V1_Omega1 = dolfinx.fem.locate_dofs_topological(V1, ct.dim, cells_Omega1)
dofs_V2_Omega2 = dolfinx.fem.locate_dofs_topological(V2, ct.dim, cells_Omega2)
dofs_M_Gamma = dolfinx.fem.locate_dofs_topological(M, ft.dim, facets_Gamma)

restriction_V1_Omega1 = multiphenicsx.fem.DofMapRestriction(V1.dofmap, dofs_V1_Omega1)
restriction_V2_Omega2 = multiphenicsx.fem.DofMapRestriction(V2.dofmap, dofs_V2_Omega2)

restriction_M_Gamma = multiphenicsx.fem.DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V1_Omega1, restriction_V2_Omega2, restriction_M_Gamma]

In [10]:
# Define trial and test functions
(u1, u2, l) = (ufl.TrialFunction(V1), ufl.TrialFunction(V2), ufl.TrialFunction(M))
(v1, v2, m) = (ufl.TestFunction(V1), ufl.TestFunction(V2), ufl.TestFunction(M))

In [11]:
# Define problem block forms
zero = dolfinx.fem.Constant(domain, petsc4py.PETSc.ScalarType(0))
Vcell = dolfinx.fem.Constant(domain, petsc4py.PETSc.ScalarType(1))
a = [
    [ufl.inner(ufl.grad(u1), ufl.grad(v1)) * dx(markers.electrolyte), None, ufl.inner(l("-"), v1("-")) * dS(markers.electrolyte_v_positive_am)],
    [None, ufl.inner(ufl.grad(u2), ufl.grad(v2)) * dx(markers.positive_am), - ufl.inner(l("+"), v2("+")) * dS(markers.electrolyte_v_positive_am)],
    [ufl.inner(u1("-"), m("-")) * dS(markers.electrolyte_v_positive_am), - ufl.inner(u2("+"), m("+")) * dS(markers.electrolyte_v_positive_am), None],
]
f = [
    ufl.inner(zero, v1) * dx(markers.electrolyte),
    ufl.inner(zero, v2) * dx(markers.positive_am),
    ufl.inner(zero, v1) * ds(markers.insulated_electrolyte) + ufl.inner(zero, v1) * ds(markers.insulated_positive_am)
]
a_cpp = dolfinx.fem.form(a)
f_cpp = dolfinx.fem.form(f)

In [12]:
# Define boundary conditions
left_dofs = fem.locate_dofs_topological(V1, ft.dim, ft.find(markers.left))
right_dofs = fem.locate_dofs_topological(V2, ft.dim, ft.find(markers.right))
dofs_V1_partial_Omega = dolfinx.fem.locate_dofs_topological(
    V1, ft.dim, facets_partial_Omega)
dofs_V2_partial_Omega = dolfinx.fem.locate_dofs_topological(
    V2, ft.dim, facets_partial_Omega)
bc_left = fem.dirichletbc(zero, left_dofs, V1)
bc_right = fem.dirichletbc(Vcell, right_dofs, V2)
bcs = [bc_left, bc_right]

In [13]:
# Assemble the block linear system
A = multiphenicsx.fem.petsc.assemble_matrix_block(a_cpp, bcs=bcs, restriction=(restriction, restriction))
A.assemble()
F = multiphenicsx.fem.petsc.assemble_vector_block(f_cpp, a_cpp, bcs=bcs, restriction=restriction)

In [14]:
# Solve
u1u2l = multiphenicsx.fem.petsc.create_vector_block(f_cpp, restriction=restriction)
ksp = petsc4py.PETSc.KSP()
ksp.create(domain.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.getPC().setFactorSetUpSolverType()
ksp.getPC().getFactorMatrix().setMumpsIcntl(icntl=7, ival=4)
ksp.setFromOptions()
ksp.solve(F, u1u2l)
u1u2l.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
ksp.destroy()

In [15]:
# Split the block solution in components
(u1, u2, l) = (dolfinx.fem.Function(V1), dolfinx.fem.Function(V2), dolfinx.fem.Function(M))
with multiphenicsx.fem.petsc.BlockVecSubVectorWrapper(
        u1u2l, [V1.dofmap, V2.dofmap, M.dofmap], restriction) as u1u2l_wrapper:
    for u1u2l_wrapper_local, component in zip(u1u2l_wrapper, (u1, u2, l)):
        with component.x.petsc_vec.localForm() as component_local:
            component_local[:] = u1u2l_wrapper_local
u1u2l.destroy()

In [17]:
with VTXWriter(comm, potential_resultsfile, [u1], engine="BP5") as fp:
    fp.write(0.0)

In [19]:
viskex.dolfinx.plot_scalar_field(u1, "u1", show_edges=False)

Widget(value='<iframe src="http://localhost:45407/index.html?ui=P_0x71949f8e4a10_3&reconnect=auto" class="pyvi…

In [20]:
viskex.dolfinx.plot_scalar_field(u2, "u2", show_edges=False)

Widget(value='<iframe src="http://localhost:45407/index.html?ui=P_0x71949f2e8110_4&reconnect=auto" class="pyvi…